# Convergence to endpoints with high probability for LASSO 

#### Alexei Stepanenko

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F
from scipy.stats import uniform_direction, uniform, bernoulli
from tqdm import tqdm

Consider the unconstrained LASSO problem
$$
\textrm{argmin}_{w\in \mathbb{R}^d} \|X w - y\|_2^2 + \|w\|_1
$$
where $y \in \mathbb{R}^m$ and $X \in \mathbb{R}^{m \times d}$ are defined by $y = e_1$ and 
$$
X = (\alpha \quad 1 \quad \cdots \quad  1  ) \,\oplus \, (I_{m-1} \quad 0_{d - m - 1})
$$
for some $\alpha > 0$, where there are $N$ ones. 

The solution always satisfies $w_3 = \cdots = w_d = 0$ so we can consider for simplicity the problem
$$
\textrm{argmin}_{w_1, w_2\in \mathbb{R}} |\alpha w_1 + w_2 + \cdots + w_N - 1|^2 + \alpha |w_1| + \cdots + |w_N|.
$$
We focus on the case $\alpha = 1$, in which case the solution is the convex hull of $\{\tfrac{1}{2} e_1, ...,\tfrac{1}{2} e_N\}$.  Furthermore, the solution can become either one of endpoints (shifted slightly) with an arbitrarily small perturbation of $\alpha$ away from $1$.

**Experiment:** Fix $\epsilon\geq0$. Run the ISTA algorithm until convergence (until the loss is less than $ 3/4 + \epsilon$), with $(w_1,..., w_N)$ initialised according to a uniform distribution in a $N-1$ dimensional sphere  $\partial B_R(0)$.

In [ ]:
def shrink_op(x,lr):
    'Shrinkage operator for ISTA'
    return F.relu(abs(x) - lr)*torch.sign(x)

def train_ISTA_conv(w, alpha = 1, ep = 10**(-4), lr = 0.1, max_steps = 10000000):
    '''
    Returns ISTA training paths for initial condition `w` (one dimensional torch tensor)
    See https://www.ceremade.dauphine.fr/~carlier/FISTA , eq. 1.4 and 1.5.
    `ep`: tolerance. `lr`: learning rate. `max_steps`: max number of steps
    '''
    w = w.clone().detach().requires_grad_(True)
    loss_hist = []
    w_hist = [w.detach().numpy()]  # list of torch tensors
    N = len(w)
    X = torch.eye(N)
    X[0,0] = alpha
    Xw = X @ w
    n = 0
    loss = (Xw.sum() - 1)**2 + abs(Xw).sum()
    while loss.item() >= 3/4 + ep and n <= max_steps:
        Xw = X @ w
        loss_noreg = (Xw.sum() - 1)**2 
        loss = loss_noreg + abs(Xw).sum()
        loss_hist.append(loss.item())
        loss_noreg.backward()
        w.data = shrink_op(w.data - lr * w.grad.data, lr)
        w.grad.data.zero_()
        w_hist.append(w.detach().numpy())
        n += 1
    if n > max_steps:
        print('TERMINATED DUE TO RUNTIME')
    return np.array(w_hist), loss_hist

def distCorner(w, N):
    "\ell^2 distance of w (nd array) to convex hull of {0.5*e_1,...,0.5*e_N}"
    dists_to_corners = []
    for i in range(N):
        corner  = np.zeros(N)
        corner[i] = 0.5
        dists_to_corners.append(np.linalg.norm(corner - w))
    return min(dists_to_corners)
        
    

#### Testing: reproducing past results

In [ ]:
w_hist, loss_hist = train_ISTA_conv(torch.tensor([2,1], dtype = torch.float32))

In [ ]:
w_hist = w_hist.transpose()
plt.plot(w_hist[0],w_hist[1])


In [ ]:
plt.plot(loss_hist)
plt.plot([0,len(loss_hist)],[3/4,3/4])

In [ ]:
R = 2; num_samp = 1000; tol = 0.01

w0_samps = R*uniform_direction.rvs(dim=2, size=num_samp)
w_final_samps = []
for w0 in w0_samps:
    w_final, train_hist = train_ISTA_conv(torch.tensor(w0, dtype = torch.float32))
    w_final = w_final[-1]
    w_final_samps.append(w_final)
distEnd_samps = [distCorner(w, 2) for w in w_final_samps]
mean_distEnd = np.array(distEnd_samps).mean()
std_distEnd = np.array(distEnd_samps).std()

pos_lst = []
neg_lst = []
for i in range(len(w0_samps)):
    if distEnd_samps[i] < tol:
        pos_lst.append(np.array(w0_samps[i]))
    else:
        neg_lst.append(np.array(w0_samps[i]))
pos_lst = np.array(pos_lst).transpose()
neg_lst = np.array(neg_lst).transpose()
plt.figure(figsize = (5,5))
plt.scatter(pos_lst[0], pos_lst[1], label= 'Converge to endpoint')
plt.scatter(neg_lst[0], neg_lst[1], label= 'Dont converge to endpoint')
plt.plot([1/2,0],[0,1/2], color = 'black', lw = 2)
plt.xlabel(r'$w_1$')
plt.ylabel(r'$w_2$')
plt.legend(loc = 1)
plt.show()

print('Mean distance to endpoint', mean_distEnd)

In [ ]:
R = 20; num_samp = 1000; tol = 0.01

w0_samps = R*uniform_direction.rvs(dim=2, size=num_samp)
w_final_samps = []
for w0 in w0_samps:
    w_final, train_hist = train_ISTA_conv(torch.tensor(w0, dtype = torch.float32))
    w_final = w_final[-1]
    w_final_samps.append(w_final)
distEnd_samps = [distCorner(w, 2) for w in w_final_samps]
mean_distEnd = np.array(distEnd_samps).mean()
std_distEnd = np.array(distEnd_samps).std()

pos_lst = []
neg_lst = []
for i in range(len(w0_samps)):
    if distEnd_samps[i] < tol:
        pos_lst.append(np.array(w0_samps[i]))
    else:
        neg_lst.append(np.array(w0_samps[i]))
pos_lst = np.array(pos_lst).transpose()
neg_lst = np.array(neg_lst).transpose()
plt.figure(figsize=(5,5))
plt.scatter(pos_lst[0], pos_lst[1], label= 'Converge to endpoint')
plt.scatter(neg_lst[0], neg_lst[1], label= 'Dont converge to endpoint')
plt.plot([1/2,0],[0,1/2], color = 'black', lw = 2)
plt.xlabel(r'$w_1$')
plt.ylabel(r'$w_2$')
plt.legend(loc = 1)
plt.show()

print('Mean distance to endpoint', mean_distEnd)

In [ ]:
R = 2; num_samp = 3000; alpha = 1

w0_samps = R*uniform_direction.rvs(dim=3, size=num_samp)
w_final_samps = []
for w0 in w0_samps:
    w_final, train_hist = train_ISTA_conv(torch.tensor(w0, dtype = torch.float32))
    w_final = w_final[-1]
    w_final_samps.append(w_final)
distEnd_samps = [distCorner(w, 3) for w in w_final_samps]
mean_distEnd = np.array(distEnd_samps).mean()
std_distEnd = np.array(distEnd_samps).std()

tol = 0.01
pos1_lst = []
pos2_lst = []
pos3_lst = []
neg_lst = []
for i in range(len(w0_samps)):
    if distEnd_samps[i] < tol:
        if abs(w_final_samps[i][0] - 1/2) < tol:
            pos1_lst.append(np.array(w0_samps[i]))
        if abs(w_final_samps[i][1] - 1/2) < tol:
            pos2_lst.append(np.array(w0_samps[i]))
        if abs(w_final_samps[i][2] - 1/2) < tol:
            pos3_lst.append(np.array(w0_samps[i]))
    else:
        neg_lst.append(np.array(w0_samps[i]))
pos1_lst = np.array(pos1_lst).transpose()
pos2_lst = np.array(pos2_lst).transpose()
pos3_lst = np.array(pos3_lst).transpose()
neg_lst = np.array(neg_lst).transpose()
fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(projection='3d')
ax.scatter(pos1_lst[0], pos1_lst[1], pos1_lst[2], label= 'Converge to endpoint 1')
ax.scatter(pos2_lst[0], pos2_lst[1], pos2_lst[2], label= 'Converge to endpoint 2')
ax.scatter(pos3_lst[0], pos3_lst[1], pos3_lst[2], label= 'Converge to endpoint 3')
ax.scatter(neg_lst[0], neg_lst[1], neg_lst[2], label= 'Dont converge to endpoint')
plt.xlabel(r'$w_1$')
plt.ylabel(r'$w_2$')
plt.ylabel(r'$w_3$')
plt.legend(loc = 1)
plt.show()

print('Mean distance to endpoint', mean_distEnd)
print('Proportion converging to e_1', len(pos1_lst[0])/len(w0_samps))
print('Proportion converging to e_2', len(pos2_lst[0])/len(w0_samps))
print('Proportion converging to e_3', len(pos3_lst[0])/len(w0_samps))

In [ ]:
R = 10; num_samp = 3000; alpha = 1

w0_samps = R*uniform_direction.rvs(dim=3, size=num_samp)
w_final_samps = []
for w0 in w0_samps:
    w_final, train_hist = train_ISTA_conv(torch.tensor(w0, dtype = torch.float32))
    w_final = w_final[-1]
    w_final_samps.append(w_final)
distEnd_samps = [distCorner(w, 3) for w in w_final_samps]
mean_distEnd = np.array(distEnd_samps).mean()
std_distEnd = np.array(distEnd_samps).std()

tol = 0.01
pos1_lst = []
pos2_lst = []
pos3_lst = []
neg_lst = []
for i in range(len(w0_samps)):
    if distEnd_samps[i] < tol:
        if abs(w_final_samps[i][0] - 1/2) < tol:
            pos1_lst.append(np.array(w0_samps[i]))
        if abs(w_final_samps[i][1] - 1/2) < tol:
            pos2_lst.append(np.array(w0_samps[i]))
        if abs(w_final_samps[i][2] - 1/2) < tol:
            pos3_lst.append(np.array(w0_samps[i]))
    else:
        neg_lst.append(np.array(w0_samps[i]))
pos1_lst = np.array(pos1_lst).transpose()
pos2_lst = np.array(pos2_lst).transpose()
pos3_lst = np.array(pos3_lst).transpose()
neg_lst = np.array(neg_lst).transpose()
fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(projection='3d')
ax.scatter(pos1_lst[0], pos1_lst[1], pos1_lst[2], label= 'Converge to endpoint 1')
ax.scatter(pos2_lst[0], pos2_lst[1], pos2_lst[2], label= 'Converge to endpoint 2')
ax.scatter(pos3_lst[0], pos3_lst[1], pos3_lst[2], label= 'Converge to endpoint 3')
ax.scatter(neg_lst[0], neg_lst[1], neg_lst[2], label= 'Dont converge to endpoint')
plt.xlabel(r'$w_1$')
plt.ylabel(r'$w_2$')
plt.ylabel(r'$w_3$')
plt.legend(loc = 1)
plt.show()

print('Mean distance to endpoint', mean_distEnd)
print('Proportion converging to e_1', len(pos1_lst[0])/len(w0_samps))
print('Proportion converging to e_2', len(pos2_lst[0])/len(w0_samps))
print('Proportion converging to e_3', len(pos3_lst[0])/len(w0_samps))

#### Rate of conv. as $R \to \infty$ for varying $N$ 

**To do:** Add error bars by repeating experiment multiple times and computing mean/std

In [ ]:
def give_probs_sphere(R, num_samp, alpha, N, tol = 0.01):
    'Gives probability convergence to endpoint and point e_1 upon random initialisation on sphere of radius `R`'
    w0_samps = R*uniform_direction.rvs(dim=N, size=num_samp)
    w_final_samps = []
    for w0 in w0_samps:
        w_final, train_hist = train_ISTA_conv(torch.tensor(w0, dtype = torch.float32), alpha = alpha)
        w_final = w_final[-1]
        w_final_samps.append(w_final)
    distEnd_samps = [distCorner(w, N) for w in w_final_samps]
    e1 = np.zeros(N); e1[0] = 1
    dist_e1_samps = [np.linalg.norm(w - 0.5*e1) for w in w_final_samps]
    num_1 = 0
    num_pos = 0
    for i in range(num_samp):
        if distEnd_samps[i] < tol:
            num_pos += 1
            if dist_e1_samps[i] < tol:
                num_1 += 1
    prob_end = num_pos/num_samp
    prob_1 = num_1/num_samp
    return prob_end, prob_1

We look at final distance to an endpoint and to $e_1$ as $R \to \infty$ for varying $N$ (fix $\alpha =1$)

In [ ]:
num_samp  = 5000
R_vals = np.linspace(3,20,12)
N_vals =[2,3,5,8]
alpha = 1

end_vals_all = []; e1_vals_all = []
for N in N_vals:
    end_vals = []; e1_vals = []
    for R in tqdm(R_vals):
        prob_end, prob_1 = give_probs_sphere(R, num_samp, alpha, N)
        end_vals.append(prob_end); e1_vals.append(prob_1)
    end_vals_all.append(end_vals); e1_vals_all.append(e1_vals)

fig, ax = plt.subplots(1,2, figsize= (24,10))
for i in range(len(N_vals)):
    label = 'N = ' + str(N_vals[i])
    ax[0].plot(R_vals,end_vals_all[i], label = label)
ax[0].set_ylabel('Probability of convergence to endpoint')
ax[0].set_xlabel('R')
ax[0].legend()
ax[0].plot([R_vals[0],R_vals[-1]],[1,1],linestyle ='--', color = 'gray')
for i in range(len(N_vals)):
    label = 'N = ' + str(N_vals[i])
    ax[1].plot(R_vals,e1_vals_all[i], label = label)
ax[1].set_ylabel('Probability of convergence to $e_1$')
ax[1].set_xlabel('R')
ax[1].legend()
for N in N_vals:
    ax[1].plot([R_vals[0],R_vals[-1]],[1/N,1/N],linestyle ='--', color = 'gray')
plt.show()



We look at final distance to an endpoint and to $e_1$ as $R \to \infty$ for varying $\alpha$ (fix $N = 2$)

In [ ]:
num_samp  = 5000
R_vals = np.linspace(3,40,12)
N = 2
al_vals = [0.9,0.99, 0.999, 1, 1.001, 1.01]

end_vals_all = []; e1_vals_all = []
for alpha in al_vals:
    end_vals = []; e1_vals = []
    for R in tqdm(R_vals):
        prob_end, prob_1 = give_probs_sphere(R, num_samp, alpha, N)
        end_vals.append(prob_end); e1_vals.append(prob_1)
    end_vals_all.append(end_vals); e1_vals_all.append(e1_vals)

fig, ax = plt.subplots(1,2, figsize= (24,10))
for i in range(len(al_vals)):
    label = 'alpha = ' + str(al_vals[i])
    ax[0].plot(R_vals,end_vals_all[i], label = label)
ax[0].set_ylabel('Probability of convergence to endpoint')
ax[0].set_xlabel('R')
ax[0].legend()
ax[0].plot([R_vals[0],R_vals[-1]],[1,1],linestyle ='--', color = 'gray')
for i in range(len(al_vals)):
    label = 'alpha = ' + str(al_vals[i])
    ax[1].plot(R_vals,e1_vals_all[i], label = label)
ax[1].set_ylabel('Probability of convergence to $e_1$')
ax[1].set_xlabel('R')
ax[1].legend()
ax[1].plot([R_vals[0],R_vals[-1]],[0,0],linestyle ='--', color = 'gray')
ax[1].plot([R_vals[0],R_vals[-1]],[1/2,1/2],linestyle ='--', color = 'gray')
ax[1].plot([R_vals[0],R_vals[-1]],[1,1],linestyle ='--', color = 'gray')
plt.show()


I don't run $\alpha = 1.1$ as it takes way too long to converge for some reason.  

### Bonus: Plotting trajectories

In [ ]:
R = 5; tol = 0.01

w0_samps = R*np.array([[np.cos(th), np.sin(th)] for th in np.linspace(0,2*np.pi,1000)])
w_final_samps = []
trainhist_samps = []
for w0 in w0_samps:
    w_final, train_hist = train_ISTA_conv(torch.tensor(w0, dtype = torch.float32))
    trainhist_samps.append(w_final)
    w_final = w_final[-1]
    w_final_samps.append(w_final)
distEnd_samps = [distCorner(w, 2) for w in w_final_samps]

pos_lst = []; neg_lst = []
poshist_lst = []; neghist_lst = []
for i in range(len(w0_samps)):
    if distEnd_samps[i] < tol:
        pos_lst.append(np.array(w0_samps[i]))
        poshist_lst.append(trainhist_samps[i])
    else:
        neg_lst.append(np.array(w0_samps[i]))
        neghist_lst.append(trainhist_samps[i])
pos_lst = np.array(pos_lst).transpose()
neg_lst = np.array(neg_lst).transpose()
plt.figure(figsize = (11,11))
plt.scatter(pos_lst[0], pos_lst[1], label= 'Converge to endpoint', color = 'green' )
plt.scatter(neg_lst[0], neg_lst[1], label= 'Dont converge to endpoint', color = 'red')
for traj in poshist_lst:
    traj = np.array(traj).transpose()
    plt.plot(traj[0], traj[1], color = 'green', alpha = 0.1)
for traj in neghist_lst:
    traj = np.array(traj).transpose()
    plt.plot(traj[0], traj[1], color = 'red', alpha = 0.1)
plt.plot([1/2,0],[0,1/2], color = 'black', lw = 2)
plt.xlabel(r'$w_1$')
plt.ylabel(r'$w_2$')
plt.legend(loc = 1)
plt.show()